In [2]:
import numpy as np
from scipy.optimize import linprog

# Define the cost matrix from the table (Billy Bishop = row 1, Pearson = row 2)
cost_matrix = np.array([
    [0.05, 0.06, 0.07, 0.08, 0.09, 0.10],  # Billy Bishop costs
    [0.08, 0.05, 0.09, 0.10, 0.07, 0.06]   # Toronto Pearson costs
])

# Expand cost matrix for all 29 sites (repeating nearest values for remaining sites)
cost_per_dose = []
for j in range(29):
    cost_per_dose.append(cost_matrix[0, min(j, 5)])  # Billy Bishop costs
for j in range(29):
    cost_per_dose.append(cost_matrix[1, min(j, 5)])  # Toronto Pearson costs

# Total number of decision variables (2 airports * 29 sites)
num_vars = 58  

# Equality constraints matrix and RHS
A_eq = np.zeros((2, num_vars))  # 2 equality constraints for total vaccine distribution
b_eq = [100000, 250000]  # Total vaccines from each airport

# Populate equality constraint for Billy Bishop (first 29 variables)
A_eq[0, :29] = 1  # Sum of x_1j should be 100,000

# Populate equality constraint for Pearson (next 29 variables)
A_eq[1, 29:] = 1  # Sum of x_2j should be 250,000

# Bounds: Each variable (dose count) must be non-negative
bounds = [(0, None)] * num_vars

# Solve LP using linprog (minimization)
result = linprog(cost_per_dose, A_eq=A_eq, b_eq=b_eq, bounds=bounds, method='highs')

# Extract the optimal transportation cost
optimal_transportation_cost = result.fun

# Print the result
print("Optimal Transportation Cost for the Week: $", round(optimal_transportation_cost, 2))


Optimal Transportation Cost for the Week: $ 17500.0


In [7]:

import gurobipy as gp
from gurobipy import GRB

# Parameters
n_sites = 29       # Total vaccination sites
n_airports = 2     # 1: Billy Bishop, 2: Toronto Pearson

# Create a new model
m = gp.Model("VaccineTransportation")

# Create decision variables x[i,j] for i = 0 (Airport 1) and 1 (Airport 2) and j = 0,...,28 (Sites)
x = {}
for i in range(n_airports):
    for j in range(n_sites):
        # Continuous variables: doses transported from airport i to site j
        x[i, j] = m.addVar(lb=0, name="x_%d_%d" % (i+1, j+1))
m.update()

# Define cost coefficients (per dose) as assumed:
# For Billy Bishop (i = 0):
#   - For hospital sites (sites 1-7, j=0..6): cost = 0.06 dollars
#   - For city-run sites (sites 8-29, j=7..28): cost = 0.07 dollars
# For Toronto Pearson (i = 1):
#   - For hospital sites (j=0..6): cost = 0.08 dollars
#   - For city-run sites (j=7..28): cost = 0.09 dollars
cost = {}
for j in range(n_sites):
    if j < 7:
        cost[0, j] = 0.06
    else:
        cost[0, j] = 0.07
for j in range(n_sites):
    if j < 7:
        cost[1, j] = 0.08
    else:
        cost[1, j] = 0.09

# Set the objective: minimize total transportation cost
m.setObjective(gp.quicksum(cost[i, j] * x[i, j] for i in range(n_airports) for j in range(n_sites)), GRB.MINIMIZE)

# --------------------
# Supply constraints (doses available at each airport)
# Billy Bishop (Airport 1, i=0): total doses = 100,000
m.addConstr(gp.quicksum(x[0, j] for j in range(n_sites)) == 100000, "Supply_Airport1")
# Toronto Pearson (Airport 2, i=1): total doses = 250,000
m.addConstr(gp.quicksum(x[1, j] for j in range(n_sites)) == 250000, "Supply_Airport2")

# --------------------
# Demand (or capacity) constraints for each vaccination site
# For hospital sites (sites 1-7, j=0..6): each receives 28,000 doses per week.
# For city-run sites (sites 8-29, j=7..28): each receives 7,000 doses per week.
for j in range(n_sites):
    if j < 7:
        cap = 28000
    else:
        cap = 7000
    m.addConstr(x[0, j] + x[1, j] == cap, "Demand_site_%d" % (j+1))

# --------------------
# Additional restrictions:
# 1. For sites 1-5 (j=0..4), the difference between doses from the two airports must be within 4,800.
m.addConstr(gp.quicksum(x[0, j] for j in range(5)) - gp.quicksum(x[1, j] for j in range(5)) <= 4800, "AbsDiff_Pos")
m.addConstr(gp.quicksum(x[1, j] for j in range(5)) - gp.quicksum(x[0, j] for j in range(5)) <= 4800, "AbsDiff_Neg")

# 2. For sites 21-25 (j=20..24): doses from Toronto Pearson (i=1) must be <= 8 times doses from Billy Bishop (i=0) for sites 11-15 (j=10..14).
m.addConstr(gp.quicksum(x[1, j] for j in range(20, 25)) <= 8 * gp.quicksum(x[0, j] for j in range(10, 15)), "Restriction_2")

# 3. For sites 26-29 (j=25..28): doses from Billy Bishop must be >= 0.8 times doses from Toronto Pearson for sites 16-20 (j=15..19).
m.addConstr(gp.quicksum(x[0, j] for j in range(25, 29)) >= 0.8 * gp.quicksum(x[1, j] for j in range(15, 20)), "Restriction_3")

# --------------------
# Optimize the model
m.optimize()

# Output the optimal transportation cost
print("Optimal Transportation Cost for the Week: $", m.objVal)


Set parameter Username
Set parameter LicenseID to value 2609998
Academic license - for non-commercial use only - expires 2026-01-14
Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 35 rows, 58 columns and 155 nonzeros
Model fingerprint: 0xb67ac3d5
Coefficient statistics:
  Matrix range     [8e-01, 8e+00]
  Objective range  [6e-02, 9e-02]
  Bounds range     [0e+00, 0e+00]
  RHS range        [5e+03, 2e+05]
Presolve removed 32 rows and 53 columns
Presolve time: 0.00s
Presolved: 3 rows, 5 columns, 9 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    2.5034952e+04   1.565655e+04   0.000000e+00      0s
       3    2.7540000e+04   0.000000e+00   0.000000e+00      0s

Solved in 3 iterations and 0.02 seconds (0.00 work units)
Optimal objective  2.754000000e+04
Optimal Transportation Cost for the Week: $ 2754

In [ ]:


#导入路径

df = pd.read_csv('/Users/Sam/Downloads/sp500_data.csv')


In [8]:
import pandas as pd

# Load the CSV file using the specified path
df = pd.read_csv('/Users/Sam/Downloads/sp500_data.csv')

# Display the first few rows to verify the data
print(df.head())

# Filter for companies whose 'Location of Headquarters' contains "New York"
ny_companies = df[df["Location of Headquarters"].str.contains("New York", case=False, na=False)]

# Count the number of companies headquartered in New York City
num_ny_companies = ny_companies.shape[0]
print("Number of companies headquartered in New York City:", num_ny_companies)


  Ticker symbol                   Company                  GICS Sector  \
0           AEE               Ameren Corp                    Utilities   
1           AXP       American Express Co                   Financials   
2             T                  AT&T Inc  Telecommunications Services   
3           AVP             Avon Products             Consumer Staples   
4           BFB  Brown-Forman Corporation             Consumer Staples   

  Location of Headquarters  Price  PercentReturn  
0      St. Louis, Missouri  31.26           4.83  
1       New York, New York  49.23           1.53  
2            Dallas, Texas  30.09           5.82  
3       New York, New York  18.31           5.26  
4     Louisville, Kentucky  82.28           1.74  
Number of companies headquartered in New York City: 11


In [9]:
import pandas as pd

# Load the CSV file using the specified path
df = pd.read_csv('/Users/Sam/Downloads/sp500_data.csv')

# Display the first few rows to verify the data
print(df.head())

# Filter for companies whose 'Location of Headquarters' exactly equals "New York, New York"
ny_companies = df[df["Location of Headquarters"].str.strip() == "New York, New York"]

# Count the number of companies headquartered in New York City
num_ny_companies = ny_companies.shape[0]
print("Number of companies headquartered in New York City:", num_ny_companies)


  Ticker symbol                   Company                  GICS Sector  \
0           AEE               Ameren Corp                    Utilities   
1           AXP       American Express Co                   Financials   
2             T                  AT&T Inc  Telecommunications Services   
3           AVP             Avon Products             Consumer Staples   
4           BFB  Brown-Forman Corporation             Consumer Staples   

  Location of Headquarters  Price  PercentReturn  
0      St. Louis, Missouri  31.26           4.83  
1       New York, New York  49.23           1.53  
2            Dallas, Texas  30.09           5.82  
3       New York, New York  18.31           5.26  
4     Louisville, Kentucky  82.28           1.74  
Number of companies headquartered in New York City: 6


In [10]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Load the data file
df = pd.read_csv('/Users/Sam/Downloads/sp500_data.csv')
n = len(df)  # number of companies (should be 67)

# Create a new Gurobi model
m = gp.Model("InvestmentPortfolio")

# Create decision variables: x[i] is the dollar amount invested in company i
x = {}
for i in range(n):
    x[i] = m.addVar(lb=0, name="x_%d" % (i+1))
m.update()

# Objective: maximize the total expected return over 1 year
# Expected return (in dollars) for company i = x[i]*(PercentReturn[i]/100)
m.setObjective(gp.quicksum((df.loc[i, "PercentReturn"] / 100.0) * x[i] for i in range(n)), GRB.MAXIMIZE)

# Constraint: Total investment is $10 million
m.addConstr(gp.quicksum(x[i] for i in range(n)) == 10000000, "TotalInvestment")

# Constraint: At most $600,000 can be invested in any individual stock
for i in range(n):
    m.addConstr(x[i] <= 600000, "MaxInvestment_%d" % (i+1))

# Constraint: No more than $500,000 can be invested in the Telecommunications sector
telecom_indices = [i for i in range(n) if df.loc[i, "GICS Sector"].strip().lower() == "telecommunications services"]
m.addConstr(gp.quicksum(x[i] for i in telecom_indices) <= 500000, "TelecomLimit")

# Constraint: Investment in the Information Technology (IT) sector must be at least 75% of the amount invested in Telecommunications
it_indices = [i for i in range(n) if df.loc[i, "GICS Sector"].strip().lower() == "information technology"]
m.addConstr(gp.quicksum(x[i] for i in it_indices) >= 0.75 * gp.quicksum(x[i] for i in telecom_indices), "IT_Telecom_Ratio")

# Constraint: The absolute difference between the total invested in Consumer Discretionary and Consumer Staples should not exceed $200,000
cd_indices = [i for i in range(n) if df.loc[i, "GICS Sector"].strip().lower() == "consumer discretionary"]
cs_indices = [i for i in range(n) if df.loc[i, "GICS Sector"].strip().lower() == "consumer staples"]
m.addConstr(gp.quicksum(x[i] for i in cd_indices) - gp.quicksum(x[i] for i in cs_indices) <= 200000, "CD_CS_Diff_Pos")
m.addConstr(gp.quicksum(x[i] for i in cs_indices) - gp.quicksum(x[i] for i in cd_indices) <= 200000, "CD_CS_Diff_Neg")

# Constraint: At least $1,000,000 must be invested in the Energy sector
energy_indices = [i for i in range(n) if df.loc[i, "GICS Sector"].strip().lower() == "energy"]
m.addConstr(gp.quicksum(x[i] for i in energy_indices) >= 1000000, "EnergyInvestment")

# Constraint: At least $300,000 must be invested in companies headquartered in New York, New York
nyc_indices = [i for i in range(n) if df.loc[i, "Location of Headquarters"].strip() == "New York, New York"]
m.addConstr(gp.quicksum(x[i] for i in nyc_indices) >= 300000, "NYCInvestment")

# Optimize the model
m.optimize()

# Print the optimal expected return (objective value)
print("Optimal Expected Return after 1 year: $", m.objVal)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 74 rows, 67 columns and 196 nonzeros
Model fingerprint: 0x4b090aa3
Coefficient statistics:
  Matrix range     [8e-01, 1e+00]
  Objective range  [2e-03, 1e-01]
  Bounds range     [0e+00, 0e+00]
  RHS range        [2e+05, 1e+07]
Presolve removed 68 rows and 16 columns
Presolve time: 0.00s
Presolved: 6 rows, 52 columns, 92 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    1.4560000e+06   3.474937e+06   0.000000e+00      0s
       6    5.1346000e+05   0.000000e+00   0.000000e+00      0s

Solved in 6 iterations and 0.01 seconds (0.00 work units)
Optimal objective  5.134600000e+05
Optimal Expected Return after 1 year: $ 513460.0


In [11]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('/Users/Sam/Downloads/Routes.csv')

# Display the first few rows to ensure it loaded correctly
print(df.head())

# We assume the CSV has columns like 'RouteID', 'Start', 'End', and 'Cost'
# If the column names differ, adjust accordingly.
# Find the row with the maximum cost
max_cost_row = df.loc[df['Cost'].idxmax()]

# Print the route with the largest cost
print("Route with the largest cost:")
print(max_cost_row)


                     Routes  Cost
0  [U, a, b, f, e, d, c, U]   136
1     [U, a, b, f, e, d, U]   110
2        [U, a, b, f, e, U]    90
3     [U, a, b, f, d, c, U]   132
4     [U, a, b, f, d, e, U]   118
Route with the largest cost:
Routes    [U, d, c, a, b, f, e, U]
Cost                           138
Name: 34, dtype: object


In [13]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# 1. Load the data file
#    The CSV has columns: "Routes" and "Cost"
df = pd.read_csv('/Users/Sam/Downloads/Routes.csv')

# 2. Create a Gurobi model
m = gp.Model("FindMaxCostRoute")

# 3. Add a binary decision variable for each row in the DataFrame
#    We'll use the row index as the key in our dictionary
x = {}
for i in df.index:
    x[i] = m.addVar(vtype=GRB.BINARY, name=f"x_{i}")
m.update()

# 4. Objective: select exactly ONE route, maximizing its Cost
m.setObjective(
    gp.quicksum(df.loc[i, "Cost"] * x[i] for i in df.index),
    GRB.MAXIMIZE
)

# 5. Constraint: exactly one route is chosen
m.addConstr(gp.quicksum(x[i] for i in df.index) == 1, "SelectOneRoute")

# 6. Optimize
m.optimize()

# 7. Identify the chosen route and print it
chosen_index = None
for i in df.index:
    if x[i].X > 0.5:  # means x[i] = 1
        chosen_index = i
        break

if chosen_index is not None:
    chosen_route = df.loc[chosen_index, "Routes"]
    chosen_cost = df.loc[chosen_index, "Cost"]
    print(f"Route with the largest cost: {chosen_route}, Cost = {chosen_cost}")
else:
    print("No route was selected (check constraints).")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 1 rows, 58 columns and 58 nonzeros
Model fingerprint: 0x06a07352
Variable types: 0 continuous, 58 integer (58 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [7e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 1e+00]
Found heuristic solution: objective 136.0000000
Presolve removed 1 rows and 58 columns
Presolve time: 0.00s
Presolve: All rows and columns removed

Explored 0 nodes (0 simplex iterations) in 0.01 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 2: 138 136 

Optimal solution found (tolerance 1.00e-04)
Best objective 1.380000000000e+02, best bound 1.380000000000e+02, gap 0.0000%
Route with the largest cost: [U, d, c, a, b, f, e, U], Cost = 138


In [14]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Load the data file (58 routes, each with a 'Routes' description and a 'Cost')
df = pd.read_csv('/Users/Sam/Downloads/Routes.csv')

# Create a new Gurobi model
m = gp.Model("NoSubsidy_RouteSelection")

# Create a binary decision variable for each route
x = {}
for i in df.index:
    x[i] = m.addVar(vtype=GRB.BINARY, name=f"x_{i}")
m.update()

# Objective: Minimize total cost WITHOUT the $50 subsidy
# That means the cost is exactly df.loc[i, 'Cost'] for each route
m.setObjective(
    gp.quicksum(df.loc[i, "Cost"] * x[i] for i in df.index),
    GRB.MINIMIZE
)

# ADD YOUR PROBLEM-SPECIFIC CONSTRAINTS HERE
# For example:
# 1) If Glendon is visited by more than one shuttle, add extra $350 to the cost
#    => This is typically modeled with a big-M or additional binary variable approach
# 2) Must serve all required campuses/stops, etc.
#    => Possibly coverage constraints or flow constraints
# ...
# m.addConstr(...)
# m.addConstr(...)

# Optimize the model
m.optimize()

# After solving, count how many routes are selected
num_selected = sum(x[i].X for i in df.index if x[i].X > 0.5)
print(f"Number of routes selected with NO subsidy: {int(num_selected)}")


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 0 rows, 58 columns and 0 nonzeros
Model fingerprint: 0x5953d813
Variable types: 0 continuous, 58 integer (58 binary)
Coefficient statistics:
  Matrix range     [0e+00, 0e+00]
  Objective range  [7e+01, 1e+02]
  Bounds range     [1e+00, 1e+00]
  RHS range        [0e+00, 0e+00]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Number of routes selected with NO subsidy: 0


In [16]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Load the CSV, making the first column (e.g. "Food_1", "Food_2", ...) as the row index
df = pd.read_csv('/Users/Sam/Downloads/nutrient_content.csv', index_col=0)

# The row index now holds the food names. Let's extract them:
foods = df.index  # This will be something like ["Food_1", "Food_2", "Food_3", ...]

# Create a new Gurobi model
m = gp.Model("DietProblem")

# Create one continuous decision variable per food item
# Each variable x[food] represents the quantity of that food in the diet
x = {}
for food in foods:
    x[food] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"x_{food}")

m.update()

# Print the number and type of decision variables
print("Number of decision variables:", len(x))
print("Type of decision variables: Continuous (non-negative)")

# (Optional) You can add constraints or an objective as needed:
# e.g., an objective to minimize cost, or constraints on nutrient intake, etc.
# m.setObjective(...)
# m.addConstr(...)
# m.optimize()


Number of decision variables: 120
Type of decision variables: Continuous (non-negative)


In [19]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Example: minimal diet model
m = gp.Model("DietProblem")

# Suppose we have 3 foods for simplicity
foods = ["Food_1", "Food_2", "Food_3"]

# Decision variables (non-negativity is built in, so not counted as constraints)
x = {}
for food in foods:
    x[food] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"x_{food}")

# Update the model so variables are recognized
m.update()

# Add some example constraints (nutrient minimums, etc.)
# Constraint 1: e.g., total quantity of Food_1 + Food_2 >= 10
m.addConstr(x["Food_1"] + x["Food_2"] >= 10, name="NutrientMin1")

# Constraint 2: e.g., total quantity of Food_2 + Food_3 <= 20
m.addConstr(x["Food_2"] + x["Food_3"] <= 20, name="NutrientMax2")

# Constraint 3: e.g., total daily calories must be exactly 2000
# (Here we assume Food_1 has 100 cal/unit, Food_2 has 50 cal/unit, Food_3 has 200 cal/unit)
m.addConstr(100*x["Food_1"] + 50*x["Food_2"] + 200*x["Food_3"] == 2000, name="CaloriesExact")

# Set an objective (for illustration, minimize total cost)
# Suppose cost: Food_1 = $2, Food_2 = $1.5, Food_3 = $3
m.setObjective(2*x["Food_1"] + 1.5*x["Food_2"] + 3*x["Food_3"], GRB.MINIMIZE)

# Optimize
m.optimize()

# (b) How many constraints are in the LP not including non-negativity?
# Gurobi property: m.NumConstrs returns the number of constraints added via addConstr()
print("Number of constraints (excluding non-negativity):", m.NumConstrs)

# Explanation:
# - Non-negativity is part of variable definitions (lb=0),
#   so it is not counted in NumConstrs.
# - NumConstrs only counts the constraints you explicitly added with addConstr().


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 3 columns and 7 nonzeros
Model fingerprint: 0x82f2fac4
Coefficient statistics:
  Matrix range     [1e+00, 2e+02]
  Objective range  [2e+00, 3e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e+01, 2e+03]
Presolve time: 0.00s
Presolved: 3 rows, 3 columns, 7 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    3.0000000e+01   8.000000e+01   0.000000e+00      0s
       1    3.5000000e+01   0.000000e+00   0.000000e+00      0s

Solved in 1 iterations and 0.00 seconds (0.00 work units)
Optimal objective  3.500000000e+01
Number of constraints (excluding non-negativity): 3


In [17]:
print("Number of constraints:", m.NumConstrs)


Number of constraints: 0


In [18]:
print("Number of constraints:", m.getAttr("NumConstrs"))


Number of constraints: 0


In [20]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# Load the nutrient content data, using the first column as the food names
df = pd.read_csv('/Users/Sam/Downloads/nutrient_content.csv', index_col=0)
foods = df.index  # This should give a list of food items

# Create a new Gurobi model for the diet problem
m = gp.Model("DietVariety")

# Create continuous decision variables x[food] representing the quantity (e.g., grams) of each food
x = {}
for food in foods:
    x[food] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"x_{food}")

# Create binary variables y[food] that indicate whether food 'food' is included in the diet
y = {}
# Choose a sufficiently large constant M (this should be large enough so that if the food is selected, 
# the maximum possible quantity can be achieved; adjust as needed)
M = 10000  
for food in foods:
    y[food] = m.addVar(vtype=GRB.BINARY, name=f"y_{food}")

m.update()

# Linking constraints: if a food is not selected (y[food] = 0) then x[food] must be 0.
for food in foods:
    m.addConstr(x[food] <= M * y[food], name=f"Link_{food}")

# Variety constraint: require that at least L different foods are selected
# For example, set L to 10 (meaning at least 10 distinct foods are used)
L = 10
m.addConstr(gp.quicksum(y[food] for food in foods) >= L, name="VarietyConstraint")

# (Optional) Set an objective (e.g., minimize total quantity, or minimize cost if cost data is available)
# Here we simply minimize the total quantity, for illustration purposes.
m.setObjective(gp.quicksum(x[food] for food in foods), GRB.MINIMIZE)

m.optimize()

# Print the number of different foods selected
num_foods_selected = sum(y[food].X for food in foods)
print("Number of distinct foods selected:", int(num_foods_selected))


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 121 rows, 240 columns and 360 nonzeros
Model fingerprint: 0x54125f16
Variable types: 120 continuous, 120 integer (120 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+04]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+01, 1e+01]
Found heuristic solution: objective 0.0000000

Explored 0 nodes (0 simplex iterations) in 0.03 seconds (0.00 work units)
Thread count was 1 (of 8 available processors)

Solution count 1: 0 

Optimal solution found (tolerance 1.00e-04)
Best objective 0.000000000000e+00, best bound 0.000000000000e+00, gap 0.0000%
Number of distinct foods selected: 120


In [21]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# -----------------------------
# 1. LOAD THE DATA
# -----------------------------
# Nutrient content: index_col=0 so that the first column is the "Food" index
df_nutrients = pd.read_csv('/Users/Sam/Downloads/nutrient_content.csv', index_col=0)
foods = df_nutrients.index  # e.g. ["Food_1", "Food_2", ...]
nutrients = df_nutrients.columns  # e.g. ["Nutrient_1", "Nutrient_2", ...]

# Nutrient requirements
df_req = pd.read_csv('/Users/Sam/Downloads/nutrient_requirements.csv')
# Suppose columns are ["Nutrient", "MinRequirement", "MaxRequirement"]

# Food cost / preference info
df_cost = pd.read_csv('/Users/Sam/Downloads/food_preferences.csv')
# Suppose columns are ["Food", "Cost"]
cost_map = dict(zip(df_cost["Food"], df_cost["Cost"]))

# Food categories
df_cat = pd.read_csv('/Users/Sam/Downloads/food_categories.csv')
# Suppose columns are ["Food", "Category"]
# We want sets S_K = { foods in category K }
categories = df_cat["Category"].unique()
cat_map = {K: set() for K in categories}
for _, row in df_cat.iterrows():
    cat_map[row["Category"]].add(row["Food"])

# Suppose we also have a dictionary: V_K for each category K
# e.g. V = {"Vegetarian": 1000, "Vegan": 500, "Kosher": 1200, "Halal": 800, "All": 0}
# You might store it in a CSV or define it manually

V = {
    "Vegetarian": 1000,
    "Vegan": 500,
    "Kosher": 1200,
    "Halal": 800,
    "All": 0  # or some required min for "All"
}

# -----------------------------
# 2. CREATE THE MODEL
# -----------------------------
m = gp.Model("OptiDiet")

# Create one continuous decision variable x[food] for each food
x = {}
for food in foods:
    x[food] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"x_{food}")

m.update()

# -----------------------------
# 3. OBJECTIVE: MINIMIZE COST
# -----------------------------
# total cost = sum_{food} cost[food] * x[food]
m.setObjective(
    gp.quicksum(cost_map[food] * x[food] for food in foods),
    GRB.MINIMIZE
)

# -----------------------------
# 4. CONSTRAINTS
# -----------------------------

# (A) Nutrient constraints
# For each nutrient n in df_req, ensure min and/or max
for _, row in df_req.iterrows():
    nutrient_name = row["Nutrient"]
    min_req = row["MinRequirement"]
    max_req = row["MaxRequirement"]  # If some are NaN or no limit, handle accordingly

    # sum_{food} (nutrient_content[food, nutrient_name] * x[food]) >= min_req
    m.addConstr(
        gp.quicksum(df_nutrients.loc[food, nutrient_name] * x[food] for food in foods) >= min_req,
        name=f"Min_{nutrient_name}"
    )

    # If there's a valid maximum, add it
    if not pd.isna(max_req) and max_req > 0:
        m.addConstr(
            gp.quicksum(df_nutrients.loc[food, nutrient_name] * x[food] for food in foods) <= max_req,
            name=f"Max_{nutrient_name}"
        )

# (B) Category constraints
# sum_{food in category K} x[food] >= V[K]
for K in V:
    if V[K] > 0:
        m.addConstr(
            gp.quicksum(x[f] for f in cat_map[K] if f in foods) >= V[K],
            name=f"Category_{K}"
        )

# (C) Variety constraint: no single item > 3% of total
# sumAll = sum_{food} x[food]
sumAll = gp.quicksum(x[food] for food in foods)
for food in foods:
    m.addConstr(x[food] <= 0.03 * sumAll, name=f"Variety_{food}")

# -----------------------------
# 5. SOLVE THE MODEL
# -----------------------------
m.optimize()

# -----------------------------
# 6. PRINT RESULTS
# -----------------------------
if m.status == GRB.OPTIMAL:
    print("Optimal solution found!")
    print(f"Optimal Food Production Cost: ${m.objVal:,.2f}")
    for food in foods:
        qty = x[food].X
        if qty > 1e-6:
            print(f"{food}: {qty:.2f} units")
else:
    print("No optimal solution found. Gurobi status code:", m.status)


KeyError: 'Food'

In [22]:
import gurobipy as gp
from gurobipy import GRB
import pandas as pd

# 1. Load the CSV, making the first column the row index
df = pd.read_csv('/Users/Sam/Downloads/nutrient_content.csv', index_col=0)

# 2. Create a Gurobi model
m = gp.Model("DietProblem")

# 3. Create decision variables: x[food], representing how much of each food is chosen
x = {}
for food in df.index:
    x[food] = m.addVar(lb=0, vtype=GRB.CONTINUOUS, name=f"x_{food}")
m.update()

# 4. Objective: For this example, assume all foods have the same 'cost' of 1
#    so we minimize the total quantity. Adjust if you have an actual cost column.
m.setObjective(gp.quicksum(x[food] for food in df.index), GRB.MINIMIZE)

# 5. Nutrient constraints
#    According to your screenshot, the CSV columns are exactly:
#    "Nutrient_1", "Nutrient_2", "Nutrient_3"
nutrients = ["Nutrient_1", "Nutrient_2", "Nutrient_3"]

# Example: set some minimum requirements for each nutrient
# (You would replace these with your actual minimum or maximum constraints)
min_reqs = {
    "Nutrient_1": 0.01,
    "Nutrient_2": 0.02,
    "Nutrient_3": 0.03
}

for nutrient in nutrients:
    m.addConstr(
        gp.quicksum(df.loc[food, nutrient] * x[food] for food in df.index) >= min_reqs[nutrient],
        name=f"Min_{nutrient}"
    )

# 6. Optimize
m.optimize()

# 7. Check solution
if m.status == GRB.OPTIMAL:
    print("Optimal objective value:", m.objVal)
    # Print which foods are used in the solution
    for food in df.index:
        if x[food].X > 1e-6:
            print(f"{food}: {x[food].X:.4f}")
else:
    print("No optimal solution found. Gurobi status code:", m.status)


Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (mac64[arm] - Darwin 24.3.0 24D70)

CPU model: Apple M3
Thread count: 8 physical cores, 8 logical processors, using up to 8 threads

Optimize a model with 3 rows, 120 columns and 360 nonzeros
Model fingerprint: 0xfd5938a5
Coefficient statistics:
  Matrix range     [1e-03, 1e-02]
  Objective range  [1e+00, 1e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-02, 3e-02]
Presolve removed 0 rows and 10 columns
Presolve time: 0.00s
Presolved: 3 rows, 110 columns, 330 nonzeros

Iteration    Objective       Primal Inf.    Dual Inf.      Time
       0    0.0000000e+00   4.800000e-01   0.000000e+00      0s
       2    3.0585594e+00   0.000000e+00   0.000000e+00      0s

Solved in 2 iterations and 0.01 seconds (0.00 work units)
Optimal objective  3.058559355e+00
Optimal objective value: 3.0585593547048067
Food_29: 0.7212
Food_116: 2.3373
